In [5]:
#反向投影，也就是说，与ROI有关的都会比较明显
#使用统计学的语言，BackProjection中存储的数值
#代表了测试图像中该像素属于皮肤区域的概率。以上图为例，亮的区域是皮肤区域的可能性更大，而暗的区域则表示更低的可能性。
import cv2
import numpy as np
from matplotlib import pyplot as plt


def nothing(x):
    pass


img = cv2.imread('../images/hand.jpg')

ROI = img[300:400,600:700]
cv2.imshow('ROI Image', ROI)

imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
roiHSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)
cv2.namedWindow('BackProjection Image')
cv2.createTrackbar('H Value', 'BackProjection Image', 0, 180, nothing)

while(1):

    hSize = cv2.getTrackbarPos('H Value', 'BackProjection Image')
    hSize = max(2, hSize)
    roiHSVHist = cv2.calcHist([roiHSV], [0, 1], None, [hSize, 256], [0, 180, 0, 256])
    cv2.normalize(roiHSVHist, roiHSVHist, 0, 255, cv2.NORM_MINMAX)
    backProjImg = cv2.calcBackProject([imgHSV], [0, 1], roiHSVHist, [0, 180, 0, 256], 1)
    cv2.imshow('BackProjection Image', backProjImg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

cv2.waitKey()

-1

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# 读入两幅不同的图像 验证模板匹配的精确度
img1 = cv2.imread('../images/jump.jpg')
# 读入模板
temp = cv2.imread('../images/jump_temp.jpg')
imgGray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# 做一次拷贝 在循环中用得到
imgGrayCopy = imgGray.copy()
tempGray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
h, w = tempGray.shape
# 对模板阈值化
_, tempGray = cv2.threshold(tempGray, 200, 255, cv2.THRESH_BINARY)
cv2.imshow('temp', tempGray)

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']



for meth in methods:
   # 每次循环都使用一次拷贝版本，防止重复画矩形
   imgGray = imgGrayCopy.copy()
   method = eval(meth)
   res = cv2.matchTemplate(imgGray, tempGray, method)
   # 定位匹配坐标
   min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
   # 确定矩形框
   if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
      top_left = min_loc
   else:
      top_left = max_loc
   bottom_right = (top_left[0] + w, top_left[1] + h)
   cv2.rectangle(imgGray, top_left, bottom_right, 0, 3)
    
   plt.figure(figsize=(15,10))
   plt.subplot(121), plt.imshow(res, cmap='gray')
   plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
   plt.subplot(122), plt.imshow(imgGray, cmap='gray')
   plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
   plt.suptitle(meth)

   plt.show()

cv2.waitKey()
cv2.destroyAllWindows()